# Azure AutoML Fair & Interpretable Model
We are using credit risk modelling dataset from Kaggle for the task. In this project, we train an AutoML model using Azure AutoML and after selecting the model,
 we assess it's fairness and interpretablity.

In [3]:
import os
from azureml.core import Workspace, Datastore, Dataset


In [14]:
# Get Workspace
ws = Workspace.from_config()

datastore_name = 'rohands'
storage_account_name = 'rohansa'
container_name ='rohan-blob'

# Get Datastore otherwise create it
if datastore_name in ws.datastores:
    blob_ds = Datastore.get(ws, datastore_name=datastore_name)
else:
    try:
        blob_ds = Datastore.register_azure_blob_container(
            workspace=ws,
            datastore_name=datastore_name,
            account_name=storage_account_name,
            container_name=container_name
        )
    except Exception as ex:
        print(ex)

# Set as Default Datastore
default_ds = ws.set_default_datastore(datastore_name)
default_ds = ws.get_default_datastore()

# Get Dataset otherise create it
dataset_name = 'credit risk dataset'

if dataset_name not in ws.datasets:
    default_ds.upload_files(
        files=['./data/credit_risk_dataset.csv'],
        target_path='/azure/azure-automl-fair-interpret-model/',
        overwrite=True,
        show_progress=True)
    
    tab_dataset = Dataset.Tabular.from_delimited_files(
        path=(default_ds,'/azure/azure-automl-fair-interpret-model/data/*.csv'))
        
    try:
        tab_dataset = tab_dataset.register(workspace=ws, 
                                name=dataset_name,
                                description='credit risk data from Kaggle',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    tab_dataset = Dataset.get(ws, dataset_name=dataset_name)
    print('Dataset already registered.')

UserErrorException: UserErrorException:
	Message: './data/credit_risk_dataset.csv' does not point to a file. Please upload the file to cloud first if running in a cloud notebook.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "'./data/credit_risk_dataset.csv' does not point to a file. Please upload the file to cloud first if running in a cloud notebook."
    }
}